# Using Dask with Amazon SageMaker

In [1]:
!pip install dask

In [2]:
!free -th

             total       used       free     shared    buffers     cached
Mem:          3.9G       1.6G       2.3G       520K        80M       446M
-/+ buffers/cache:       1.1G       2.8G
Swap:           0B         0B         0B
Total:        3.9G       1.6G       2.3G


In [3]:
from dask.distributed import LocalCluster, Client
cluster = LocalCluster()
client = Client(cluster)



In [5]:
import dask.dataframe as df


In [6]:
%time dfdaskfeb = df.read_csv('./taxiapril.csv')

CPU times: user 28.7 ms, sys: 12.2 ms, total: 40.9 ms
Wall time: 38.5 ms


In [7]:
%time dfdaskfeb.head(5)


CPU times: user 70.8 ms, sys: 13.6 ms, total: 84.4 ms
Wall time: 1.95 s


,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
0,HV0005,B02510,2019-04-01 00:11:25,2019-04-01 00:21:56,52,97,NaN
1,HV0005,B02510,2019-04-01 00:28:57,2019-04-01 00:52:24,97,239,NaN
2,HV0005,B02510,2019-04-01 00:59:25,2019-04-01 01:29:54,238,61,NaN
3,HV0005,B02510,2019-04-01 00:53:14,2019-04-01 00:57:02,212,213,1.0
4,HV0003,B02764,2019-04-01 00:11:42,2019-04-01 00:20:06,165,210,1.0


In [8]:
!free -th

             total       used       free     shared    buffers     cached
Mem:          3.9G       1.9G       2.0G       568K        80M       508M
-/+ buffers/cache:       1.3G       2.6G
Swap:           0B         0B         0B
Total:        3.9G       1.9G       2.0G


In [9]:
%time dfdaskapril = df.read_csv('./taxifeb.csv')

CPU times: user 29.6 ms, sys: 0 ns, total: 29.6 ms
Wall time: 34.6 ms


In [10]:
dfdaskapril.head(5)

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
0,HV0003,B02867,2019-02-01 00:05:18,2019-02-01 00:14:57,245,251,NaN
1,HV0003,B02879,2019-02-01 00:41:29,2019-02-01 00:49:39,216,197,NaN
2,HV0005,B02510,2019-02-01 00:51:34,2019-02-01 01:28:29,261,234,NaN
3,HV0005,B02510,2019-02-01 00:03:51,2019-02-01 00:07:16,87,87,NaN
4,HV0005,B02510,2019-02-01 00:09:44,2019-02-01 00:39:56,87,198,NaN


In [11]:
!free -th

             total       used       free     shared    buffers     cached
Mem:          3.9G       1.9G       1.9G       568K        80M       569M
-/+ buffers/cache:       1.3G       2.5G
Swap:           0B         0B         0B
Total:        3.9G       1.9G       1.9G


In [12]:
%time dfjoin = df.concat([dfdaskfeb,dfdaskapril], axis=0)

CPU times: user 28.3 ms, sys: 3.95 ms, total: 32.2 ms
Wall time: 31.1 ms


In [13]:
dfjoin.head(5)
print(len(dfjoin))


41893869


In [15]:
%time dfclean= dfdaskapril.fillna(value=0)
dfclean.head(5)

CPU times: user 8.63 ms, sys: 0 ns, total: 8.63 ms
Wall time: 7.85 ms


,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
0,HV0003,B02867,2019-02-01 00:05:18,2019-02-01 00:14:57,245,251,0.0
1,HV0003,B02879,2019-02-01 00:41:29,2019-02-01 00:49:39,216,197,0.0
2,HV0005,B02510,2019-02-01 00:51:34,2019-02-01 01:28:29,261,234,0.0
3,HV0005,B02510,2019-02-01 00:03:51,2019-02-01 00:07:16,87,87,0.0
4,HV0005,B02510,2019-02-01 00:09:44,2019-02-01 00:39:56,87,198,0.0


In [16]:
%time dfclean.groupby('PULocationID').count().compute()

CPU times: user 733 ms, sys: 130 ms, total: 863 ms
Wall time: 20 s


,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,DOLocationID,SR_Flag
PULocationID,,,,,,
1,1314,1314,1314,1314,1314,1314
2,13,13,13,13,13,13
3,33923,33923,33923,33923,33923,33923
4,94427,94427,94427,94427,94427,94427
5,3955,3955,3955,3955,3955,3955
...,...,...,...,...,...,...
265,1278,1278,1278,1278,1278,1278
110,12,12,12,12,12,12
264,14,14,14,14,14,14
